# Overview


Data Source:

1. Energy Data: https://www.kaggle.com/datasets/lislejoem/us_energy_census_gdp_10-14?resource=download

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from bs4 import BeautifulSoup
import requests

# Extract

## Store CSV into DataFrame

In [ ]:
#Load in energy data
csv_file_1 = "resources/energy_census_and_economic_data_US_2010-2014.csv"
energy_df = pd.read_csv(csv_file_1)
energy_df.head()

# Transform

In [ ]:
#Cleaning Energy Consumption Data
year = []
state = []
energy_consumption = []
gdp = []
#Sectors of energy consumption:
biomass_consumption = []
coal_consumption = []
elec_consumption = []
fossfuel_consumption = []
natgas_consumption = []

for index, row in energy_df.iterrows():
    year.append('2010')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2010'][index])
    gdp.append(energy_df['GDP2010'][index])
    biomass_consumption.append(energy_df['BiomassC2010'][index])
    coal_consumption.append(energy_df['CoalC2010'][index])
    elec_consumption.append(energy_df['ElecC2010'][index])
    fossfuel_consumption.append(energy_df['FossFuelC2010'][index])
    natgas_consumption.append(energy_df['NatGasC2010'][index])
    
for index, row in energy_df.iterrows():
    year.append('2011')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2011'][index])
    gdp.append(energy_df['GDP2011'][index])
    biomass_consumption.append(energy_df['BiomassC2011'][index])
    coal_consumption.append(energy_df['CoalC2011'][index])
    elec_consumption.append(energy_df['ElecC2011'][index])
    fossfuel_consumption.append(energy_df['FossFuelC2011'][index])
    natgas_consumption.append(energy_df['NatGasC2011'][index])
    
for index, row in energy_df.iterrows():
    year.append('2012')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2012'][index])
    gdp.append(energy_df['GDP2012'][index])
    biomass_consumption.append(energy_df['BiomassC2012'][index])
    coal_consumption.append(energy_df['CoalC2012'][index])
    elec_consumption.append(energy_df['ElecC2012'][index])
    fossfuel_consumption.append(energy_df['FossFuelC2012'][index])
    natgas_consumption.append(energy_df['NatGasC2012'][index])
    
for index, row in energy_df.iterrows():
    year.append('2013')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2013'][index])
    gdp.append(energy_df['GDP2013'][index])
    biomass_consumption.append(energy_df['BiomassC2013'][index])
    coal_consumption.append(energy_df['CoalC2013'][index])
    elec_consumption.append(energy_df['ElecC2013'][index])
    fossfuel_consumption.append(energy_df['FossFuelC2013'][index])
    natgas_consumption.append(energy_df['NatGasC2013'][index])
    
for index, row in energy_df.iterrows():
    year.append('2014')
    state.append(energy_df['State'][index])
    energy_consumption.append(energy_df['TotalC2014'][index])
    gdp.append(energy_df['GDP2014'][index])
    biomass_consumption.append(energy_df['BiomassC2014'][index])
    coal_consumption.append(energy_df['CoalC2014'][index])
    elec_consumption.append(energy_df['ElecC2014'][index])
    fossfuel_consumption.append(energy_df['FossFuelC2014'][index])
    natgas_consumption.append(energy_df['NatGasC2014'][index])

In [ ]:
energy_cleaned_df = pd.DataFrame({
#     'year_state': year_state,
    'year': year,
    'state': state,
    'total_energy': energy_consumption,
    'gdp': gdp,
    'biomass_consumption': biomass_consumption,
    'coal_consumption': coal_consumption,
    'electricity_consumption': elec_consumption,
    'fossil_fuel_consumption': fossfuel_consumption,
    'natural_gas_consumption': natgas_consumption
})

energy_cleaned_df

In [ ]:
#Scraping state coordinates
url = 'https://inkplant.com/code/state-latitudes-longitudes'

table_name = 'table table-hover' 
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

coordinates_table = soup.find('table', {'class':table_name})

df = pd.read_html(str(coordinates_table))

state_coordinates_df = df[0]

In [ ]:
# coordinates = []
# for index, row in state_coordinates_df.iterrows():
#     coordinate = []
#     x = state_coordinates_df['Longitude'][index]
#     y = state_coordinates_df['Latitude'][index]
#     coordinate.append(x)
#     coordinate.append(y)
#     coordinates.append(coordinate)
pd.to_numeric(state_coordinates_df['Latitude'])
state_coordinates_df['Latitude'].dtype
    

In [ ]:
# state_coordinates_df['coordinates'] = coordinates
# state_coordinates_df_cleaned = state_coordinates_df[['State', 'coordinates']]
state_coordinates_df = state_coordinates_df.rename(columns = {'State': 'state', 'Latitude': 'latitude', 'Longitude': 'longitude'})
# state_coordinates_df_cleaned
state_coordinates_df.head()

In [ ]:
final_df = pd.merge(energy_cleaned_df, state_coordinates_df, how = 'left')

In [ ]:
final_df

In [ ]:
final_df.columns

## Create database connection

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'energy_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

## Load DataFrames into database

In [ ]:
final_df.to_sql(name='energy_consumption', con=engine, if_exists='append', index=False)

In [ ]:
# cleaned_min_wage_data_df.to_sql(name='minimum_wage', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the DB table

In [ ]:
pd.read_sql_query('select * from energy_consumption', con=engine).head()

In [ ]:
# Export the DataFrames to CSV

final_df.to_csv("energy_cleaned_df.csv", index=True)